In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Przygtowanie danych

In [ ]:
digits = load_digits()
X = digits.data
y = digits.target

# Normalizacja
X = X / 16.0

y_one_hot = y.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

W torch'u definiuje się klasę Dataset, która musi mieć metody __len__ i __getitem__. Później taki Dataset jest wykorzystywany przez Dataloader do ładowania danych.

In [ ]:
class DigitsDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
train_dataset = DigitsDataset(X_train, y_train)
test_dataset = DigitsDataset(X_test, y_test)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Tworzymy sieć, f-cję straty i optymalizator

nn.Module to podstawowa klasa dla sieci neuronowych. Moduły mogą zawierać inne moduły tworząc strukturę drzewiastą.

Sieć neuronowa musi mieć zaimplementowaną co najmniej metodę `forward` i konstruktor.

In [ ]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Net, self).__init__()
        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)
        self.activation = nn.Tanh()
        
    def forward(self, x):
        x = self.activation(self.hidden(x))
        x = self.output(x)
        return x

In [ ]:
input_dim = 64  # Długość wektora wejściowego
hidden_dim = 32  # Szerokość warstwy ukrytej (liczba neuronów)
output_dim = 10  # Szerokość warstwy wyjściowej (liczba możliwych klas)

model = Net(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

## Trening

- W torch'u jest moduł `Autograd`, który pozwala na automatyczne różniczkowanie.\
- Wołamy `loss.backward()` i liczone są gradienty dla wszystkich warstw w sieci.\
- `optimizer.zero_grad()` ustawia gradienty wszystkich parametrów optymalizowanych przez optimizer na zero.\
- `model.train()` i `model.eval()` dotyczą warstw, których w tym przykładzie nie używamy, ale poprawną praktyką jest używanie tych funkcji zawsze przy rozpoczęciu treningu/ewaluacji.
- `optimizer.step()` wykonuje krok optymalizacji na podstawie obliczonych gradientów i strategii optymalizatora.\

In [ ]:
epochs = 100
loss_history = []
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for X_batch, y_batch in train_loader:
        # Forward pass
        y_batch = y_batch.squeeze(1)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    loss_history.append(avg_loss)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {avg_loss:.4f}")

In [ ]:
plt.plot(range(1, epochs+1), loss_history)
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Value")

## Ewaluacja

`torch.no_grad()` wyłącza obliczanie gradientów, bo w czasie ewaluacji są niepotrzebne, redukuje to zużycie pamięci.

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        predicted = torch.argmax(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch.squeeze(1)).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")